# 反応拡散方程式

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt
import numpy as np
from numba import jit
from matplotlib import animation, rc

In [ ]:
# ラプラシアンの実装
@jit
def laplacian(m, n, s):
  ts=0.0
  ts+=s[m+1][n]
  ts+=s[m-1][n]
  ts+=s[m][n+1]
  ts+=s[m][n-1]
  # ここを埋めよ
  ts-=4*s[m][n]
  return ts

In [ ]:
# ラプラシアンのテスト
a=np.arange(9).reshape(3, 3)
a[0, 1]=0
a

In [ ]:
a=np.arange(9).reshape(3, 3)
a[0, 1]=0
laplacian(1, 1, a)

In [ ]:
# 時間発展
@jit # でコーダを忘れないこと
def calc(u, v, u2, v2):
  L, _=u.shape
  dt=0.2
  F=0.04
  k=0.06075
  Du=0.1
  Dv=0.05
  lu=np.zeros((L, L))
  lv=np.zeros((L, L))
  for ix in range(1, L-1):
    for iy in range(1, L-1):
      lu[ix, iy]=Du*laplacian(ix, iy, u)
      lv[ix, iy]=Dv*laplacian(ix, iy, v)
  cu=-v*v*u+F*(1.0-u)
  cv=v*v*u-(F+k)*v
  u2[:]=u+(lu+cu)*dt
  v2[:]=v+(lv+cv)*dt

In [ ]:
# シミュレーションループ
@jit
def simulation(L, loop):
  u=np.zeros((L, L))
  u2=np.zeros((L, L))
  v=np.zeros((L, L))
  v2=np.zeros((L, L))
  h=L//2
  u[h-6:h+6, h-6:h+6]=0.9
  v[h-3:h+3, h-3:h+3]=0.7
  r=[]
  for i in range(loop):
    calc(u, v, u2, v2)
    u, u2, v, v2=u2, u, v2, v
    if i%100==0:
      r.append(v.copy())
  return r

In [ ]:
# シミュレーションの実行
imgs=simulation(64, 10000)
n=len(imgs)
for i in range(4):
  im=plt.imshow(imgs[n//4*i])
  plt.show()

In [ ]:
# アニメーション
fig=plt.figure()
im=plt.imshow(imgs[-1])
def update(i):
  im.set_array(imgs[i])

In [ ]:
# アニメーションの表示
rc('animation', html='jshtml')
animation.FuncAnimation(fig, update, interval=50, frames=len(imgs))